# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686501


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:03<00:44,  1.58s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:52,  1.94s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:35,  1.41s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:28,  1.23s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:12<00:01,  3.85it/s]

Rendering models:  90%|█████████ | 27/30 [00:13<00:00,  3.36it/s]

Rendering models:  93%|█████████▎| 28/30 [00:13<00:00,  3.46it/s]

Rendering models:  97%|█████████▋| 29/30 [00:13<00:00,  3.34it/s]

froboy                                0.002840
rnevils                               0.001176
jmfranci                              0.002897
tosnyder                              0.001788
ncaro                                 0.002874
msalin03                              0.001156
msross                                0.001638
not-logged-in-66748d0532f903be0737    0.011422
tingard                               0.001276
AlexYoung35                           0.004063
pangeli5                              0.001663
not-logged-in-214814e52004b42491a4    0.002335
acapirala                             0.006165
jlj080498                             0.002216
not-logged-in-38467772a6aab4505762    0.002094
Lavadude                              0.001886
Mr_DMan                               0.002045
not-logged-in-15498ebaa693dd5a198d    0.001188
TateNachampassak                      0.004021
not-logged-in-fe0c6b32f06101805ea0    0.001625
DJofAwesome                           0.001596
nicmacd      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())